In [3]:
import json
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import math

#### 1e-4 entropy rewards

In [4]:
file_path = "../results/smooth_qwen3-4b/streams/actor/0/0/0.jsonl"
data = [json.loads(line)[0] for line in open(file_path)]
df = pd.DataFrame(data)

In [27]:
i = -100
print(df.iloc[i]['text'], df.iloc[i]['reward'])

<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
Let $P$ be the set of points $$\{(x, y) \mid 0 \leq x, y \leq 25, x, y \in \mathbb{Z}\}$$ and let $T$ be the set of triangles formed by picking three distinct points in $P$ (rotations, reflections, and translations count as distinct triangles). Compute the number of triangles in $T$ that have area larger than 300.<|im_end|>
<|im_start|>assistant
<think>

</think>

We are given:

- $ P = \{(x, y) \mid 0 \leq x, y \leq 25,\ x, y \in \mathbb{Z} \} $

This is a **grid of points** with integer coordinates in the square:
$$
P = \{ (x, y) \mid 0 \leq x, y \leq 25 \}
$$
So, there are **26 points along each axis**, and the total number of **points** in $ P $ is:
$$
|P| = 26 \times 26 = 676
$$

We are to count the number of **triangles** formed by **three **distinct** points in $ P $, such that the **area of the triangle is greater than 300**.

---

## Step 1: How many **triangles

#### 1e-4 entropy

In [28]:
file_path = "../results/smooth_qwen3-4b_e1e-3/streams/actor/0/0/0.jsonl"
data = [json.loads(line)[0] for line in open(file_path)]
df = pd.DataFrame(data)
i = -100
print(df.iloc[i]['text'], df.iloc[i]['reward'])

<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
Let $p>2$ be a prime number. $\mathbb{F}_{p}[x]$ is defined as the set of all polynomials in $x$ with coefficients in $\mathbb{F}_{p}$ (the integers modulo $p$ with usual addition and subtraction), so that two polynomials are equal if and only if the coefficients of $x^{k}$ are equal in $\mathbb{F}_{p}$ for each nonnegative integer $k$. For example, $(x+2)(2 x+3)=2 x^{2}+2 x+1$ in $\mathbb{F}_{5}[x]$ because the corresponding coefficients are equal modulo 5 . Let $f, g \in \mathbb{F}_{p}[x]$. The pair $(f, g)$ is called compositional if $$f(g(x)) \equiv x^{p^{2}}-x$$ in $\mathbb{F}_{p}[x]$. Find, with proof, the number of compositional pairs (in terms of $p$ ).<|im_end|>
<|im_start|>assistant
<think>

</think>

We are given:

- $ p > 2 $ is a **prime**.
- $ \mathbb{F}_p[x] $ is the ring of polynomials with **coefficients in the finite field** $ \mathbb{F}_p = \mathbb{Z}/

#### 0 entropy

In [ ]:
file_path = "../results/smooth_qwen3-4b_e0/streams/actor/0/0/0.jsonl"
data = [json.loads(line)[0] for line in open(file_path)]
df = pd.DataFrame(data)

In [29]:
i = -1
print(df.iloc[i]['text'], df.iloc[i]['reward'])

<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
On a chessboard, a queen attacks every square it can reach by moving from its current square along a row, column, or diagonal without passing through a different square that is occupied by a chess piece. Find the number of ways in which three indistinguishable queens can be placed on an $8 \times 8$ chess board so that each queen attacks both others.<|im_end|>
<|im_start|>assistant
<think>

</think>

We are asked:

> **Find the number of ways in which three **indistinguishable** queens can be placed on an $8 \times 8$ chessboard**  
> **such that each queen attacks the other two.**

---

## ✅ **Important Definitions and Concepts**

### ✅ **Queen's Movement in Chess**
- A **queen** can move **any number of squares**:
  - **Horizontally (same row)**
  - **Vertically (same column)**
  - **Diagonally (same diagonal, in any direction)**

So, a queen **attacks all squares in i

In [22]:
file_path = "../results/smooth_qwen3-4b_e0/streams/actor_test/0/0/0.jsonl"
data = [json.loads(line)[-1] for line in open(file_path)]
df = pd.DataFrame(data)

In [30]:
i = -2
print(df.iloc[i]['text'], df.iloc[i]['reward'])

<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
Let $a_0,a_1,a_2,\dots $ be a sequence of real numbers such that $a_0=0, a_1=1,$ and for every $n\geq 2$ there exists $1 \leq k \leq n$ satisfying \[ a_n=\frac{a_{n-1}+\dots + a_{n-k}}{k}. \]Find the maximum possible value of $a_{2018}-a_{2017}$.<|im_end|>
<|im_start|>assistant
<think>

</think>

We are given:

- A **sequence** $ a_0, a_1, a_2, \dots $
- **Initial conditions:**  
  $$
  a_0 = 0, \quad a_1 = 1
  $$
- **Recurrence for $ n \geq 2 $:**
  $$
  a_n = \frac{a_{n-1} + a_{n-2} + \dots + a_{n-k}}{k}
  $$
  where there **exists** at least **one** integer $ k \in [1, n] $ such that **this equality holds**.

---

We are to find:

$$
\boxed{\max(a_{2018} - a_{2017})}
$$

So we want to **maximize** the difference between **two consecutive terms** of the sequence, at positions **2017** and **2018**.

---

## **Step 1: Understand the recurrence**

For **each** $ n \geq 2

In [1]:
import json
from pathlib import Path
from tqdm import tqdm
import math

# Path to your JSONL file
file_path = "../results/smooth_qwen3-4b_e1e-3/streams/actor_test/0/0/0.jsonl"


def process_text(text):
    return text.split("<|im_start|>user\n")[1].split("<|im_end|>\n<|im_start|>assistant")[0]

def extract_text_and_reward_jsonl(input_path, output_path):
    """
    Stream a big JSONL file where each line is a list of 8 entries.
    For each entry, write out {"text": ..., "reward": ...} as a new line.
    """
    with open(input_path, "r", encoding="utf-8") as fin, \
         open(output_path, "w", encoding="utf-8") as fout:

        for line in tqdm(fin, desc="Processing rows"):
            line = line.strip()
            if not line:
                continue

            # Each row is a list of entries (e.g., length 8)
            try:
                row = json.loads(line)
            except json.JSONDecodeError:
                # optionally log or skip bad lines
                continue

            row_text = None
            row_model_version = None
            rewards = []
            for entry in row:
                # Safely get fields
                text = process_text(entry.get("text"))
                model_version = entry.get("metadata")["model_version"]

                if row_text is None: 
                    row_text = text
                else: 
                    assert text == row_text
                if row_model_version is None:
                    row_model_version = model_version
                else:
                    try:
                        assert abs(row_model_version-model_version) <= 1024
                    except:
                        print(model_version, row_model_version)
                        raise AssertionError

                rewards.append(entry.get("reward"))
            
            try:
                assert len(rewards) > 0
            except:
                print(rewards)
                raise AssertionError
                
            assert row_text is not None
            assert row_model_version is not None
            out_obj = {"text": row_text, "rewards": rewards, "model_version": row_model_version}
            fout.write(json.dumps(out_obj, ensure_ascii=False) + "\n")


extract_text_and_reward_jsonl(
    file_path, 
    file_path.split(".")[0]+"_processed.jsonl"
)


Processing rows: 1565it [00:01, 807.28it/s]


In [4]:
import pandas as pd

# Load the processed JSONL file as a dataframe
df = pd.read_json(
    file_path.split(".")[0]+"_processed.jsonl",
    lines=True
)

print(f"Loaded {len(df)} rows")
print(f"\nDataframe shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()




from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("CohenQu/POPE-MIX-first_guide-no_guide-0.0-0.64-1024-verl")

# Convert to pandas DataFrame
# If the dataset has multiple splits, you might need to specify which one
# For example: dataset['train'] or dataset['validation']
if isinstance(dataset, dict):
    # If multiple splits, use the first one (usually 'train')
    split_name = list(dataset.keys())[0]
    df_hf = dataset[split_name].to_pandas()
    print(f"Dataset has splits: {list(dataset.keys())}")
    print(f"Using split: {split_name}")
else:
    df_hf = dataset.to_pandas()

df_hf['text'] = df_hf['prompt'].apply(lambda x: x[0]['content'])


print(f"\nLoaded {len(df_hf)} rows from Hugging Face")
print(f"\nDataframe shape: {df_hf.shape}")
print(f"\nColumns: {df_hf.columns.tolist()}")
print(f"\nFirst few rows:")
df_hf.head()


text_to_data_source = df_hf.groupby('text')['level'].first().to_dict()

# df['source'] = df['text'].apply(lambda x: text_to_data_source[x])


# df['all_negative'] = df['rewards'].apply(lambda x: float(sum(x) == 0.))

Loaded 1565 rows

Dataframe shape: (1565, 3)

Columns: ['text', 'rewards', 'model_version']

First few rows:
Dataset has splits: ['train', 'test']
Using split: train

Loaded 2048 rows from Hugging Face

Dataframe shape: (2048, 8)

Columns: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'level', 'problem_id', 'text']

First few rows:


In [3]:
import matplotlib.pyplot as plt
import numpy as np

# Group by model_version and source, then average all_negative
df_plot = df.groupby(['model_version', 'source'])['all_negative'].mean().reset_index()

# Pivot to have sources as columns for easier plotting
df_pivot = df_plot.pivot(index='model_version', columns='source', values='all_negative')

# Sort by model_version to ensure proper ordering for running average
df_pivot = df_pivot.sort_index()

# Plot
plt.figure(figsize=(12, 6))

# Get a colormap for different sources
colors = plt.cm.tab10(np.linspace(0, 1, len(df_pivot.columns)))

for i, source in enumerate(df_pivot.columns):
    color = colors[i]
    # Lighter color for original (reduce alpha and brightness)
    light_color = (*color[:3], 0.3)  # Lighter with transparency
    # Darker color for smoothed (full opacity)
    dark_color = tuple(c * 0.7 for c in color[:3]) + (1.0,)  # Darker version
    
    x = df_pivot.index
    y_original = df_pivot[source]
    
    # Calculate running average (window size of 3, adjust as needed)
    window_size = 10
    y_smoothed = y_original.rolling(window=window_size, center=True, min_periods=1).mean()
    
    # Plot original values in lighter color
    plt.plot(x, y_original, marker='o', color=light_color, markersize=3, 
             label=None, alpha=0.4, linewidth=1)
    
    # Plot smoothed values in darker color
    plt.plot(x, y_smoothed, marker='o', color=dark_color, markersize=4,
             label=f'{source}', linewidth=2)

plt.xlabel('Model Version')
plt.ylabel('Average all_negative')
plt.title('all_negative averaged across model_version, per source')
plt.legend(title='Source', ncol=2, fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

KeyError: 'source'